In [2]:
!pip install langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.44
    Uninstalling langchain-core-0.3.44:
      Successfully uninstalled langchain-core-0.3.44


In [3]:
import os
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

#Few Shot Prompting

**Use Case:**

* When you need to provide the model with examples before asking for output.

**When to Use?**


* When providing contextual learning to an AI model.

* When you want the AI to follow a specific answering style.

In [12]:
from langchain import FewShotPromptTemplate, LLMChain, PromptTemplate
from langchain_openai import OpenAI, ChatOpenAI

In [7]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {"input": "What is AI?", "output": "AI stands for Artificial Intelligence."},
    {"input": "Explain ML.", "output": "ML stands for Machine Learning."}
]

example_template = PromptTemplate(
    input_variables=["input", "output"],
    template="Q: {input}\nA: {output}"
)

few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_template,
    prefix="I will answer the following questions:",
    suffix="Q: {query}\nA:",
    input_variables=["query"]
)

print(few_shot_template.format(query="What is NLP?"))


I will answer the following questions:

Q: What is AI?
A: AI stands for Artificial Intelligence.

Q: Explain ML.
A: ML stands for Machine Learning.

Q: What is NLP?
A:


In [8]:
print(few_shot_template.format(query="What is NLP?"))

I will answer the following questions:

Q: What is AI?
A: AI stands for Artificial Intelligence.

Q: Explain ML.
A: ML stands for Machine Learning.

Q: What is NLP?
A:


In [10]:
print(few_shot_template.format(query='big'))

I will answer the following questions:

Q: What is AI?
A: AI stands for Artificial Intelligence.

Q: Explain ML.
A: ML stands for Machine Learning.

Q: big
A:


In [14]:
llm = OpenAI(temperature=0)

In [15]:
chain=LLMChain(llm=llm,prompt=few_shot_template)

<ipython-input-15-a06613d5a564>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=llm,prompt=few_shot_template)


In [17]:
chain_output = chain({'query':"big"})

In [18]:
print(chain_output)

{'query': 'big', 'text': ' Big is a word used to describe something that is large in size, quantity, or importance. It can also refer to something that is significant or influential.'}


In [19]:
chain_output1 = chain({'query':"What is NLP?"})

In [20]:
print(chain_output1)

{'query': 'What is NLP?', 'text': ' NLP stands for Natural Language Processing. It is a branch of AI that focuses on enabling computers to understand, interpret, and manipulate human language. This includes tasks such as speech recognition, language translation, and text analysis. NLP is used in various applications such as virtual assistants, chatbots, and sentiment analysis.'}


In [22]:
print(chain_output1['query'])

What is NLP?


In [23]:
print(chain_output1['text'])

 NLP stands for Natural Language Processing. It is a branch of AI that focuses on enabling computers to understand, interpret, and manipulate human language. This includes tasks such as speech recognition, language translation, and text analysis. NLP is used in various applications such as virtual assistants, chatbots, and sentiment analysis.


#Chain of Thought Prompt

**Chain of Thought (CoT)** is a reasoning technique in generative AI where a model is encouraged to break down complex problems into intermediate steps before arriving at the final answer. This approach improves the model’s ability to handle multi-step reasoning tasks.

In LangChain, CoT can be implemented in various ways using prompt engineering, function calling, and memory to ensure the model follows a structured reasoning path.

**Use Cases of Chain of Thought in LangChain:**


CoT reasoning is particularly useful in:

* **Mathematical and Logical Reasoning**

 * Helps in solving step-by-step math problems, finance calculations, and logical puzzles.

* **Question Answering and Conversational AI**

 * Enables better contextual understanding and memory for chatbots.

* **Coding Assistance**

 * Debugging and code generation tasks that require structured reasoning.

* **Finance and Data Analysis**

 * Used in AI models that analyze financial reports, stock trends, or automate investment strategies.

* **Medical Diagnosis & Research**

 * Helps break down symptoms into logical steps for diagnosis.



In [24]:
# Define Chain of Thought Prompt
prompt_template = PromptTemplate(
    input_variables=["question"],
    template="""You are a logical AI that follows a step-by-step reasoning process.

    Question: {question}

    Think step by step and provide the answer.
    """
)

In [25]:
# Initialize LLM Model
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [26]:
# Create LLM Chain with CoT Prompt
cot_chain = LLMChain(llm=llm, prompt=prompt_template)

In [27]:
# Run the Chain with a Sample Question
question = "If a train leaves at 8 AM and travels 60 miles per hour, how far will it travel by 11 AM?"

In [29]:
cot_response = cot_chain.invoke(question)

In [30]:
cot_response

{'question': 'If a train leaves at 8 AM and travels 60 miles per hour, how far will it travel by 11 AM?',
 'text': "Step 1: Determine the time duration of the train's travel. The train leaves at 8 AM and we want to know how far it will travel by 11 AM. So, the duration of the travel is 11 AM - 8 AM = 3 hours.\n\nStep 2: Use the formula for distance which is speed multiplied by time. The speed of the train is given as 60 miles per hour.\n\nStep 3: Multiply the speed of the train by the duration of the travel. So, 60 miles/hour * 3 hours = 180 miles.\n\nTherefore, the train will travel 180 miles by 11 AM."}

In [31]:
cot_response['question']

'If a train leaves at 8 AM and travels 60 miles per hour, how far will it travel by 11 AM?'

In [32]:
cot_response['text']

"Step 1: Determine the time duration of the train's travel. The train leaves at 8 AM and we want to know how far it will travel by 11 AM. So, the duration of the travel is 11 AM - 8 AM = 3 hours.\n\nStep 2: Use the formula for distance which is speed multiplied by time. The speed of the train is given as 60 miles per hour.\n\nStep 3: Multiply the speed of the train by the duration of the travel. So, 60 miles/hour * 3 hours = 180 miles.\n\nTherefore, the train will travel 180 miles by 11 AM."

In [33]:
print(cot_response['text'])

Step 1: Determine the time duration of the train's travel. The train leaves at 8 AM and we want to know how far it will travel by 11 AM. So, the duration of the travel is 11 AM - 8 AM = 3 hours.

Step 2: Use the formula for distance which is speed multiplied by time. The speed of the train is given as 60 miles per hour.

Step 3: Multiply the speed of the train by the duration of the travel. So, 60 miles/hour * 3 hours = 180 miles.

Therefore, the train will travel 180 miles by 11 AM.


##With Agent

In [34]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

# Define a Tool to Perform Step-wise Calculation
def simple_math_tool(question: str):
    return f"Step-by-step solution for: {question}"

In [35]:
math_tool = Tool(
    name="Math Solver",
    func=simple_math_tool,
    description="Use this tool for multi-step math problems."
)

In [36]:
# Initialize an OpenAI LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

In [37]:
# Create an Agent with CoT Reasoning
agent = initialize_agent(
    tools=[math_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

<ipython-input-37-f83b6e7806ee>:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [38]:
# Run the Agent with a Question
response = agent.run("A car travels 45 mph for 2.5 hours. How far does it travel?")



> Entering new AgentExecutor chain...
This is a simple distance = speed * time problem.
Action: Math Solver
Action Input: 45 * 2.5
Observation: Step-by-step solution for: 45 * 2.5
Thought:I now know the final answer.
Final Answer: 112.5 miles

> Finished chain.


In [39]:
print(response)

112.5 miles


#ReAct Prompt

In [54]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.utilities import SerpAPIWrapper


In [55]:
# Initialize OpenAI LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [49]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=5152f168ccbc0a8782e3f3272e32863e11e9db950d194cac1442d35840db4d46
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [45]:
SERPER_API_KEY = userdata.get('SERPER_API_KEY')

os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [50]:
# Define a tool for web search (SerpAPI)
search = SerpAPIWrapper(serpapi_api_key  = SERPER_API_KEY)
search_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for retrieving current information from the web"
)


In [56]:
# Initialize ReAct Agent
react_agent = initialize_agent(
    tools=[search_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [57]:
query = "What is the latest news about AI?"

In [58]:
response = react_agent.run(query)



> Entering new AgentExecutor chain...
I need to search the web for the most recent news about AI.
Action: Search
Action Input: "latest news about AI"
Observation: [{'title': 'Trump again implies he is a King in sharing latest AI-generated meme', 'link': 'https://www.yahoo.com/news/trump-again-implies-king-sharing-162222560.html', 'source': 'Yahoo', 'date': '15 hours ago'}, {'title': "Exclusive: Intel's new CEO plots overhaul of manufacturing and AI operations", 'link': 'https://www.reuters.com/technology/intels-new-ceo-plots-overhaul-manufacturing-ai-operations-2025-03-17/', 'source': 'Reuters', 'date': '12 hours ago', 'thumbnail': 'https://serpapi.com/searches/67d91f36728439b772661e9e/images/86c588301748a5f668e8ad3edd4242106b94b36ecebbcc8f.jpeg'}, {'title': 'People are using Google’s new AI model to remove watermarks from images', 'link': 'https://techcrunch.com/2025/03/16/people-are-using-googles-new-ai-model-to-remove-watermarks-from-images/', 'source': 'TechCrunch', 'date': '1 da

In [59]:
print(response)

Here are some of the latest news about AI:
1. "Trump again implies he is a King in sharing latest AI-generated meme" from Yahoo 15 hours ago. [Read more](https://www.yahoo.com/news/trump-again-implies-king-sharing-162222560.html)
2. "Exclusive: Intel's new CEO plots overhaul of manufacturing and AI operations" from Reuters 12 hours ago. [Read more](https://www.reuters.com/technology/intels-new-ceo-plots-overhaul-manufacturing-ai-operations-2025-03-17/)
3. "People are using Google’s new AI model to remove watermarks from images" from TechCrunch 1 day ago. [Read more](https://techcrunch.com/2025/03/16/people-are-using-googles-new-ai-model-to-remove-watermarks-from-images/)
4. "How to use ChatGPT as a personal AI research assistant" from Popular Science 1 day ago. [Read more](https://www.popsci.com/diy/how-to-use-chatgpt-as-a-personal-ai-research-assistant/)
